In [ ]:
import os
from os.path import join
import sys
import pandas as pd
import numpy as np
import scipy.signal as signal

In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows();

In [ ]:
DATA_DIR = join("..", "data")
RAW_DIR = join(DATA_DIR, "raw")

In [ ]:
df = pd.read_csv(join(RAW_DIR, "UCI_HAPT", "RawData", "acc_exp01_user01.txt"), sep=' ', header=None)

In [ ]:
df.head()

In [ ]:
def signalplot(arr):
    df = pd.DataFrame(data=arr).rename(columns={0: 'x', 1: 'y', 2: 'z'})
    molten_df = df.reset_index().melt(id_vars=['index'], value_vars=['x', 'y', 'z'])
    return (alt
        .Chart(molten_df)
        .mark_line()
        .encode(
            x='index',
            y='value',
            color='variable'
        ))

In [ ]:
signalplot(df.values)

In [ ]:
def filter3d(arr, func):
    filtered_arr_dim_0 = func(arr.T[0])
    filtered_arr_dim_1 = func(arr.T[1])
    filtered_arr_dim_2 = func(arr.T[2])
    return np.stack((filtered_arr_dim_0, filtered_arr_dim_1, filtered_arr_dim_2), axis=-1)

### Filtering
According to the procedure described by Reyes-Ortiz et al:
- median filter (to remove noise)
- 3rd order Butterworth low-pass filter with corner frequency of 20 Hz (to remove noise)
- Butterworth low-pass filter with corner frequency of 0.3 Hz (to separate into body acceleration and gravity)

In [ ]:
medfilt_arr = filter3d(df.values, lambda x: signal.medfilt(x))
signalplot(medfilt_arr)

In [ ]:
# Digital filter Wn value must be normalized between 0 and 1, 
# where 1 is the Nyquist frequency (half the sampling rate, which in our case is 50 Hz divided by 2).
butter1 = signal.butter(3, 20/(50/2), 'lowpass', analog=False, output='ba')
butter2 = signal.butter(3, 0.3/(50/2), 'lowpass', analog=False, output='ba')

In [ ]:
butter1_arr = filter3d(medfilt_arr, lambda x: signal.lfilter(b=butter1[0], a=butter1[1], x=x, axis=0))
signalplot(butter1_arr)

In [ ]:
butter2_arr = filter3d(butter1_arr, lambda x: signal.lfilter(b=butter2[0], a=butter2[1], x=x, axis=0))
signalplot(butter2_arr)